<a href="https://colab.research.google.com/github/MoChiaoKai/Kai0214/blob/main/EX0501_Echo_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [ ]:
from google.colab import userdata
from flask import Flask, request, abort
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

# 這個函式用來啟動 ngrok
def ngrok_start():
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    # 使用 pyngrok 的 connect 函式
    ngrok.connect(5000)
    # 讓 Flask 應用程式使用 ngrok
    run_with_ngrok(app)

from linebot.v3 import WebhookHandler

from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

# 配置 LINE Bot 的 access_token 和 channel secret
Configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(Configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

ngrok_start()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d21a-34-9-203-167.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:50:06] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:50:16] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:50:51] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:51:57] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:52:03] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 07:52:09] "POST /callback HTTP/1.1" 200 -


In [ ]:
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(ngrok_token) # 這一行是關鍵！